In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys; sys.executable

'/home/mmordig/reinforcement/alphageometry/LLM_finetuner/verbalization_venv3/bin/python'

In [ ]:
from enum import Enum
from pydantic import BaseModel, constr

import outlines
import torch


class Weapon(str, Enum):
    sword = "sword"
    axe = "axe"
    mace = "mace"
    spear = "spear"
    bow = "bow"
    crossbow = "crossbow"


class Armor(str, Enum):
    leather = "leather"
    chainmail = "chainmail"
    plate = "plate"


class Character(BaseModel):
    name: str #constr(max_length=10)
    age: int
    armor: Armor
    weapon: Weapon
    strength: int


# model = outlines.models.transformers("mistralai/Mistral-7B-Instruct-v0.2")
# model = outlines.models.transformers("mistralai/Mistral-7B-Instruct-v0.3")
# model = outlines.models.transformers("SweatyCrayfish/llama-3-8b-quantized")

# Construct structured sequence generator
generator = outlines.generate.json(model, Character)

# Draw a sample
rng = torch.Generator(device="cuda")
rng.manual_seed(789001)

character = generator("Give me a character description", rng=rng)

print(repr(character))
# Character(name='Anderson', age=28, armor=<Armor.chainmail: 'chainmail'>, weapon=<Weapon.sword: 'sword'>, strength=8)

character = generator("Give me an interesting character description", rng=rng)

print(repr(character))
# Character(name='Vivian Thr', age=44, armor=<Armor.plate: 'plate'>, weapon=<Weapon.crossbow: 'crossbow'>, strength=125)

In [ ]:
import trl
trl.trainer.ConstantLengthDataset
from trl.extras.dataset_formatting import conversations_formatting_function

from datasets import load_dataset, load_from_disk, DatasetDict
from transformers.pipelines.pt_utils import KeyDataset

from transformers import AutoTokenizer, AutoModelForCausalLM
import yaml
from pathlib import Path
import os

from typing import Annotated
from transformers import LogitsProcessorList, TextStreamer
import json

from LLM_finetuner.constrained_generation import GrammarLogitProcessor, StatefulGrammarLogitProcessor, GrammarType

In [ ]:
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_name = "gpt2"
model_name = "SweatyCrayfish/llama-3-8b-quantized"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# load in 8-bit
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)
from transformers import BitsAndBytesConfig
import torch
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config)

# model.cuda()

In [ ]:
import outlines
from outlines.models import Transformers, transformers
# outlines.models.transformers()
model_outlines = Transformers(model, tokenizer)




In [ ]:
from pydantic import BaseModel, constr, field_validator
from enum import Enum

from typing import List, Union

# class Triangle(BaseModel):
#     vertices: List[str]
    
# class Quadruple(BaseModel):
#     vertices: List[str]
#     @field_validator("vertices")
#     def check_length(cls, v):
#         if (len(v) != 4) or (len(set(v)) != 4):
#             raise ValueError("Must be a quadruple")
#         return v

# class GeometryCommand(BaseModel):
#     commands: List[Union[Triangle, Quadruple]]
#     @field_validator("commands")
#     def check_commands(cls, v):
#         assert len(v) == 3

# class Triangle(BaseModel):
#     p1: str
#     p2: str
#     p3: str
    
# class Quadruple(BaseModel):
#     p1: str
#     p2: str
#     p3: str
#     p4: str

class Weapon(str, Enum):
    sword = "sword"
    axe = "axe"
    mace = "mace"
    spear = "spear"
    bow = "bow"
    crossbow = "crossbow"


class Armor(str, Enum):
    leather = "leather"
    chainmail = "chainmail"
    plate = "plate"


class Character(BaseModel):
    name: constr(max_length=10)
    age: int
    armor: Armor
    weapon: Weapon
    strength: int

In [ ]:
generator = outlines.generate.json(model_outlines, Character)

In [ ]:
generator("Generate some geometry commands: ")

In [ ]:
xxx = tokenizer("one; two; one; three; ")["input_ids"]

tokenizer.batch_decode(xxx), xxx


In [ ]:
fsm.tokenizer("one; two; one; three; ")

In [ ]:
from outlines.fsm.fsm import RegexFSM, CFGGuide, CFGFSM

grammar = """
    ?start: clauses
    ?clauses: (clause_term)+
    ?clause_term: clause ";"
    ?clause: "one" | "two" | "three"
    
    %import common.WS
    %ignore WS
"""


processor = GrammarLogitProcessor(
    tokenizer, device="unspecified", grammar=grammar, 
    # grammar_type=GrammarType.GRAMMAR_TYPE_JSON
    grammar_type=GrammarType.GRAMMAR_TYPE_REGEX
)
fsm = processor.fsm
# fsm = CFGFSM(grammar, tokenizer)



state = 0
# i = 0
for i in range(len(xxx)):
    print(f"Allowed: ...{fsm.allowed_token_ids(state)[-10:]}, choosing {xxx[i]}, valid: {xxx[i] in fsm.allowed_token_ids(state)}")
    state = fsm.next_state(state, xxx[i])
    print(f"Moved to state {state}")
# state

In [ ]:
from lark import Lark
json_parser = Lark(r"""
    ?value: dict
         | list
         | string
         | SIGNED_NUMBER -> number
         | "true" -> true | "false" -> false | "null" -> null

    string: ESCAPED_STRING
    list : "[" [value ("," value)*] "]"

    dict : "{" [pair ("," pair)*] "}"
    pair : string ":" value

    %import common.ESCAPED_STRING
    %import common.SIGNED_NUMBER
    %import common.WS
    %ignore WS

    """, start='value', parser="lalr")

from lark import Transformer

from lark import Transformer

class TreeToJson(Transformer):
    def string(self, s):
        (s,) = s
        return s[1:-1]
    def number(self, n):
        (n,) = n
        return float(n)

    list = list
    pair = tuple
    dict = dict

    null = lambda self, _: None
    true = lambda self, _: True
    false = lambda self, _: False
    
text = '{"key": ["item0", "item1", 3.14, null, false]}'
tree = json_parser.parse(text)
print( tree.pretty() )
TreeToJson().transform(tree)
# _

In [ ]:
from lark import Lark
from outlines import grammars
def LarkV1(cfg_string):
    return Lark(
            cfg_string,
            parser="lalr",
            lexer="contextual",
            propagate_positions=False,
            maybe_placeholders=False,
            regex=True,
            import_paths=[grammars.GRAMMAR_PATH],
        )
json_parser = LarkV1(r"""
    ?start: clauses
    ?clauses: (clause_term)+ -> clauses
    ?clause_term: clause ";" -> clause_term
    ?clause: "one" -> one | "two" -> two | "three" -> three
    
    %import common.WS
    %ignore WS
""")
x1 = json_parser.parse(("one; two; one; three; "))
print(x1.pretty())

In [ ]:
grammar = """
    ?start: clauses
    ?clauses: (clause_term)+
    ?clause_term: clause ";"
    ?clause: "one" | "two" | "three"
    
    %import common.WS
    %ignore WS
"""

processor = GrammarLogitProcessor(
    tokenizer, device="unspecified", grammar=grammar, 
    # grammar_type=GrammarType.GRAMMAR_TYPE_JSON
    grammar_type=GrammarType.GRAMMAR_TYPE_REGEX
)
xxx = processor.tokenizer("one; two; one; three; ")["input_ids"]
# xxx = processor.tokenizer("start : ")["input_ids"]

fsm = processor.fsm
xxx, fsm.allowed_token_ids(state=0)

# fsm.allowed_token_ids(5)

processor.tokenizer.batch_decode(xxx)


In [ ]:
grammar = r"""
    ?start: value
    ?value: dict
         | list
         | string
         | SIGNED_NUMBER -> number
         | "true" -> true | "false" -> false | "null" -> null

    string: UNESCAPED_STRING
    list : "[" [value ("," value)*] "]"

    dict : "{" [pair ("," pair)*] "}"
    pair : string ":" value

    %import common.UNESCAPED_STRING
    %import common.SIGNED_NUMBER
    %import common.WS
    %ignore WS

    """
    
# processor = GrammarLogitProcessor(
#     tokenizer, device="unspecified", grammar=grammar, 
#     # grammar_type=GrammarType.GRAMMAR_TYPE_JSON
#     grammar_type=GrammarType.GRAMMAR_TYPE_REGEX
# )

xxx = processor.tokenizer('{"key": ["item0", "item1", 3.14, null, false]}')["input_ids"]

In [ ]:

state = 0
for token_id in xxx:
    next_word, *allowed_words = processor.tokenizer.convert_ids_to_tokens([token_id] + processor.fsm.allowed_token_ids(state))
    print(f"Allowed: {allowed_words}")
    print(f"Chose {next_word}")
    state = processor.advance(token_id, state)
    print(f"State: {state}")

In [ ]:
# from outlines import models
# from outlines import generate

# def add(a: int, b: int):
#     return a + b

# model = models.transformers("mistralai/Mistral-7B-v0.1")
# generator = generate.json(model, add)
# result = generator("Return two integers named a and b respectively. a is odd and b even.")

# print(add(**result))
# # 3

In [ ]:


from pydantic import BaseModel, conint, Field
class CarSpecs(BaseModel):
    seller: str
    model: str
    year: int
    price: Annotated[int, Field(strict=True, ge=0)] #conint(ge=0)
    # year: conint(ge=1900, le=2022)
    # year: int

# grammar = CarSpecs.model_json_schema()
# grammar = json.dumps(grammar)

# text = f"Generate JSON output for this schema: {grammar}"
# text = [
#     {
#         "role": "system",
#         "content": text
#     }
# ]

# tokenizer.padding_side = "left"
# text = tokenizer.apply_chat_template(text, add_generation_prompt=True, tokenize=False)

# grammar = """
#     ?start: expression

#     ?expression: term (("+" | "-") term)*

#     ?term: factor (("*" | "/") factor)*

#     ?factor: NUMBER
#            | "-" factor
#            | "(" expression ")"

#     %import common.NUMBER
# """
# text = (
#     "Alice had 4 apples and Bob ate 2. "
#   + "Write an expression for Alice's apples:"
# )
grammar = """
?start: clauses
?clauses: clause_term+
?clause_term: clause ";"
?clause: "one" | "two" | "three"
"""
# grammar = """
#     ?start: expression

#     ?expression: term (("+" | "-") term)*

#     ?term: factor (("*" | "/") factor)*

#     ?factor: NUMBER
#            | "-" factor
#            | "(" expression ")"

#     %import common.NUMBER
# """
text = "Write some text using one, two, three:"

print(f"Text: {text}")
streamer = TextStreamer(tokenizer)
grammar_logits_processor = StatefulGrammarLogitProcessor(GrammarLogitProcessor(
    tokenizer, device="unspecified", grammar=grammar, 
    # grammar_type=GrammarType.GRAMMAR_TYPE_JSON
    grammar_type=GrammarType.GRAMMAR_TYPE_REGEX
))
encoded_inputs = tokenizer(text, return_tensors="pt").to("cuda")#, padding=True)


In [ ]:
# processor.tokenizer.convert_ids_to_tokens(xxx)
processor.fsm.allowed_token_ids(state)

In [ ]:
grammar_logits_processor.fsm_states = None
grammar_logits_processor.cur_seq_len = None
print("Starting generation:")
_ = model.generate(**encoded_inputs, logits_processor=LogitsProcessorList([grammar_logits_processor]), num_beams=2 if streamer is None else 1, max_new_tokens=40, streamer=streamer)

In [ ]:
from outlines import models, generate
from outlines.models import Transformers

arithmetic_grammar = """
    ?start: expression

    ?expression: term (("+" | "-") term)*

    ?term: factor (("*" | "/") factor)*

    ?factor: NUMBER
           | "-" factor
           | "(" expression ")"

    %import common.NUMBER
"""

# model = models.transformers("WizardLM/WizardMath-7B-V1.1")
# model.tokenizer = tokenizer
generator = generate.cfg(Transformers(model, tokenizer), arithmetic_grammar)


In [ ]:
sequence = generator(
  "Alice had 4 apples and Bob ate 2. "
  + "Write an expression for Alice's apples:"
)

# pip install git+https://github.com/outlines-dev/outlines.git pycountry pyairports
print(sequence)
# (8-2)

In [ ]:
sequence = generator(
  "Alice had 4 apples and Bob ate 2. "
  + "Write an expression for Alice's apples:"
)

print(sequence)
# (8-2)

In [ ]:
with open("/home/mmordig/reinforcement/alphageometry/defs.txt", "r") as f:
    for (i, line) in enumerate(f):
        if i % 6 != 0: continue
        line = line.strip()
        # print(line)
        fun, *args= line.split()
        print(fun, len(args))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
# print(tokenizer.pad_token)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
import functools
from datasets.fingerprint import Hasher
Hasher.hash(tokenizer)
tokenizer

def ggg(a, b):
    return a + b
functools.partial(ggg, b=2)(1)
Hasher.hash(functools.partial(ggg, b=2))

In [ ]:
from transformers import GenerationConfig
GenerationConfig(aaax=1)

In [ ]:
from peft import AutoPeftModelForCausalLM
model = AutoPeftModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
Hasher.hash(model)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")


extra_tokens_file = Path(os.path.expanduser("~/reinforcement/alphageometry/assets/def-patterns-desc.yml"))
def_to_desc = yaml.safe_load(extra_tokens_file.read_text())
tokenizer.add_tokens([defn for defn in def_to_desc.keys()], special_tokens=True)
# tokenizer.tokenize("<s>[INST] ABCD is a trapezoid. E is a points such that line CB meets line AD at E. Define points G, H, F, I such that line FG and line HI are common tangents to circle centered at B with radius BD and circle centered at E with radius EC at points F, G and H, I respectively.. J is defined such that ∠JCE is equal to ∠GIF. Let K be a point such that ∠KED is equal to ∠GHC. [/INST] A B C D = [trapezoid] A B C D; E = [intersection_ll] E C B A D; F G H I = [cc_tangent] F G H I B D E C; J = [eqangle3] J C E G I F; K = [eqangle3] K E D G H C </s>[END]")

In [ ]:

from LLM_finetuner.dataset import convert_formalalphageom_to_json


dataset_dir = "/home/mmordig/reinforcement/alphageometry/LLM_finetuner/runs/verbalization/datasets/alpha_geo_small_arrow"
dataset = load_dataset(dataset_dir)

def json_converter(batch):
    return {"fl_statement": [convert_formalalphageom_to_json(x) for x in batch["fl_statement"]]}
# logger.info("Converting to JSON")
dataset["train"].map(json_converter, batched=True)

In [ ]:
import json

def convert_formalalphageom_to_json(expression, pretty=False):
    parts = expression.split("; ")
    result = []

    for part in parts:
        defines, func_call = part.split(" = ")
        func_name, *args = func_call.split()
        defines_list = defines.split()

        func_dict = {
            "fun": func_name,
            "args": args,
            "defines": defines_list
        }

        result.append(func_dict)

    # return json.dumps(result)
    return json.dumps(result, indent=2) if pretty else result

# Example usage
expression = 'A B C = ieq_triangle A B C; D E F = risos D E F; G = psquare G A E; H = intersection_cc H C G E; I J = tangent I J C B F; K = intersection_tt K J D I E A G'
def show_transformation(expression):
    print(expression)
    print("Translation:")
    json_output = convert_formalalphageom_to_json(expression, pretty=True)
    # json_output = convert_formalalphageom_to_json(expression, pretty=False)
    print(json_output)
show_transformation(expression)
# print(convert_to_json(dataset["train"][1]["fl_statement"]))

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
?accuracy

In [ ]:
from accelerate import Accelerator
# accelerator = Accelerator()
# accelerator.accumulate(model)

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
from datasets import Dataset
def gen():
    yield {"pokemon": "bulbasaur", "type": "grass"}
    yield {"pokemon": "squirtle", "type": "water"}
ds = Dataset.from_generator(gen)
ds[0]

from datasets import IterableDataset
ds = IterableDataset.from_generator(gen)
for example in ds:
    print(example)
for example in ds:
    print(example)
    
# ds = Dataset.from_dict()
ds.set_epoch
# IterableDataset.from_generator((x for x in range(10))) # not a generator function, but a generator

In [ ]:
from datasets import Dataset
ds = Dataset.from_dict({"aa": [0, 1], "bb": [2, 3]})
ds.map(lambda x: {"cc": x["aa"] + 10}, load_from_cache_file=False, keep_in_memory=False)[:]

In [ ]:
dataset
dataset["train"][0]
dataset["train"][:2]
dataset


In [ ]:
from LLM_finetuner.dataset import replace_words_with_tokens, format_question_answer_single


extra_tokens_file = Path(os.path.expanduser("~/reinforcement/alphageometry/assets/def-patterns-desc.yml"))
def_to_desc = yaml.safe_load(extra_tokens_file.read_text())
# eq_triangle, sieq_triangle, so replace by longest first
geom_tokens = sorted(list(def_to_desc.keys()), key=lambda x: -len(x))
print(geom_tokens)

dataset = dataset.map(lambda x: {"fl_statement": replace_words_with_tokens(x["fl_statement"], geom_tokens)}, num_proc=8)
dataset["train"][:2]

In [ ]:
# from transformers import DataCollatorWithPadding
?tokenizer.pad

In [ ]:
# see transformers.Trainer.evaluation_loop
# from transformers import Trainer, TrainingArguments, Seq2SeqTrainer

import sys
from LLM_finetuner.sft_finetuning import main as get_trainer
import LLM_finetuner
LLM_finetuner.sft_finetuning.TRL_USE_RICH = False

sys.argv = ["--overwrite_output_dir", "--use_peft", "--per_device_train_batch_size", "64", "--per_device_eval_batch_size", "64", "--model_name_or_path", "gpt2", "--eval_steps", "10", "--evaluation_strategy", "steps", "--max_eval_samples", "2", "--explicit_eos_str", "[END]", "--extra_tokens_file", "/home/mmordig/reinforcement/alphageometry/assets/def-patterns-desc.yml", "--output_dir", "/home/mmordig/reinforcement/alphageometry/LLM_finetuner/runs/verbalization/training/debug112/{model_name}_{max_train_samples}ex_peft{use_peft}", "--dataset_name", "/home/mmordig/reinforcement/alphageometry/LLM_finetuner/runs/verbalization/datasets/alpha_geo_processed", "--config", "/home/mmordig/reinforcement/alphageometry/LLM_finetuner/trl_sft_config.yml", "--max_train_samples", "10", "--num_train_epochs", "100000",]
sys.argv += ["--include_inputs_for_metrics"]
# sys.argv += ["--dataloader_num_workers: 0"]
trainer, resume_checkpoint = get_trainer()


In [ ]:
import wandb
def compute_metrics(eval_results):
    breakpoint()
    predictions = eval_results.predictions
    inputs = eval_results.inputs
    labels = eval_results.label_ids
    # print(predictions)
    print(inputs[0])
    print(labels[0])
    print(predictions[0])
    predictions = trainer.tokenizer.batch_decode(predictions)
    inputs = trainer.tokenizer.batch_decode(inputs)
    labels = trainer.tokenizer.batch_decode(labels)
    print(predictions)
    print(labels)
    # print(predictions.shape)
    # return {"generated_text": predictions}
    # return {"generated_text2": wandb.Table(columns=["text"], data=[[p] for p in predictions])}
    return {"predictions": wandb.Table(columns=["input", "predicted"], data=[[i, p] for (i, p) in zip(inputs, predictions)])}
    
trainer.compute_metrics = compute_metrics
trainer.evaluate()

In [ ]:
# class TrainerWithGeneration(Trainer):
#     def generate_with_model(self):
#         model = self._wrap_model(self.model, training=False, dataloader=dataloader)
#         model.eval()
        

In [ ]:
import torch
from transformers import TrainerCallback

class MakePredictionsCallback(TrainerCallback):
    def __init__(self, max_generation_length=1024):
        # self.tokenizer = tokenizer
        self.max_generation_length = max_generation_length

    def on_epoch_end(self, args, state, control, model, eval_dataloader, **kwargs):
        model.eval()
        predictions = []
        for batch in eval_dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            with torch.no_grad():
                output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=self.max_generation_length)
            predictions.extend(output)
        # model.train()
        
        control.on_lo
        return control.

In [ ]:
import torch
import torch.nn as nn
from contextlib import nullcontext

# from torchdistx.fake import fake_mode
# with fake_mode():
with nullcontext():
    model = nn.Sequential(
        nn.Conv2d(1, 20, 5),
        nn.ReLU(),
        nn.Conv2d(20, 64, 5),
        nn.ReLU()
    )

    total_activations = 0

    def count_activations_hook(module, input, output):
        global total_activations
        activations = output.numel()
        total_activations += activations
        print(f"{module.__class__.__name__} produced {activations} activations")

    for layer in model.children():
        layer.register_forward_hook(count_activations_hook)

    input_tensor = torch.randn(1, 1, 28, 28)
    model(input_tensor)
    print(f"Total activations: {total_activations}")


In [ ]:
import torch
# model_name = "meta-llama/Llama-2-7b-chat-hf"
from accelerate import init_empty_weights
model_name = "gpt2"
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model(input_ids=torch.LongTensor([1, 2, 3])[None, :])

In [ ]:
dir(model)
# model.modules[0]
next(model.children())

In [ ]:
total_activations = 0

def count_activations_hook(module, input, output):
    global total_activations
    # output.shape[0] is the batch size
    # output.numel() gives the total number of elements in the output, for all batches
    # To get activations per single input, you would divide by output.shape[0]
    activations = output.numel()
    total_activations += activations
    print(f"{module.__class__.__name__} produced {activations} activations")

# Register hook for each layer
for layer in model.children():
    layer.register_forward_hook(count_activations_hook)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
print(accelerator.state)
print(str(accelerator))
print(f"{accelerator=}")

In [ ]:
from typing import List


    
# r'''one \"two\" 'three''''
# parse_htcondor_arguments('''"3 simple arguments"''')

from accelerate import init_empty_weights


# format_htcondor_arguments



In [ ]:
import accelerate
accelerate.wait_for_everyone()

In [ ]:
tuple("$(Cluster) $(Process) $(NumRanks)".split(" "))

In [ ]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
from LLM_finetuner.utils import add_new_tokens_with_average_init


extra_tokens_file = Path(os.path.expanduser("~/reinforcement/alphageometry/assets/def-patterns-desc.yml"))
def_to_desc = yaml.safe_load(extra_tokens_file.read_text())

add_new_tokens_with_average_init(model, tokenizer, def_to_desc)


In [ ]:
# def_to_desc
tokens = list(def_to_desc.keys())
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"The following tokens are already known, ignoring them: {[token for (token, id) in zip(tokens, token_ids) if id != tokenizer.unk_token_id]}")
def_to_desc = {token: def_to_desc[token] for (token, id) in zip(tokens, token_ids) if id == tokenizer.unk_token_id}
def_to_desc

In [ ]:
dataset_dir = "/home/mmordig/reinforcement/HumbleAttemptAtGeneralAI/runs/verbalization/datasets/alpha_geo_small_processed"
raw_datasets = load_from_disk(dataset_dir)
dataset = raw_datasets["test"]

In [ ]:
# https://stackoverflow.com/questions/76446228/setting-padding-token-as-eos-token-when-using-datacollatorforlanguagemodeling-fr


model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"
# model_name_or_path = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)#, add_eos_token=True)
# inputs = "Hello world"
inputs = [
    {"role": "system", "content": "System"},
    # must be ordered as user-assistant alternating sequence
    {"role": "user", "content": "Question1"},
    {"role": "assistant", "content": "Answer1"},
    {"role": "user", "content": "Question2"},
    {"role": "assistant", "content": "Answer2"},
]
print(tokenizer.apply_chat_template(inputs, tokenize=False, add_generation_prompt=True))
print("#"*80)

inputs = [
    {"role": "system", "content": "System"},
    # must be ordered as user-assistant alternating sequence
    {"role": "user", "content": "Question1"},
    {"role": "assistant", "content": "Answer1"},
    {"role": "user", "content": "Question2"},
    # {"role": "assistant", "content": "Answer2"},
]
print(tokenizer.apply_chat_template(inputs, tokenize=False, add_generation_prompt=True))

In [ ]:
query = "### Question: Points B, A, C are defined such that triangle ABC is an equilateral triangle. Define points D, F, & E such that E, D, and F is a right angle isosceles triangle with the right angle at D. Point G is defined such that G, A, E are three consecutive vertices of a square. Circle centered at C with radius CE intersects circle centered at G with radius GE at H and E. Points I and J are defined such that line IC and line JC are the two tangents to circle centered at B with radius BF at point I and J respectively.. Define point K such that line DI and line AG are parallel. line JK perpendicular to line DI. line EK perpendicular to line AG. line JK meets line EK at the point K. ### Answer: A B C = ieq_triangle A B C; D E F = risos D E F; G = psquare G A E; H = intersection_cc H C G E; I J = tangent I J C B F; K = intersection_tt K J D I E A G;"
tokenizer.decode(tokenizer(query)["input_ids"], skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [ ]:
tokenizer.tokenize(query)
" ".join(tokenizer.tokenize(query))

In [ ]:
import io


def print_to_str(x):
    temp_out = io.StringIO()
    print(x, file=temp_out)
    return temp_out.getvalue()
print(print_to_str("ss\na"))

In [ ]:
from LLM_finetuner.question_answer_utils import get_question_answer_to_chat_formatter

formatting_func = get_question_answer_to_chat_formatter(tokenizer, text_column="text")
print(f"Example format: {formatting_func(dataset[0])}")


In [ ]:
from geometry_translation.new.question_answer_utils import extract_question_answer



text_column = "text"
print(convert_question_answer_to_chat_format(dataset[0], text_column=text_column))

In [ ]:
encoded_inputs = tokenizer(, return_tensors="pt", add_special_tokens=True)
encoded_inputs

In [ ]:
tokenizer.decode(encoded_inputs["input_ids"][0])

In [ ]:
?tokenizer

In [ ]:
import textwrap
from datasets import load_dataset

def wrapping_print(x):
    print(textwrap.fill(x))
    


# dataset_dir = "/fast/mmordig/general_ai_rl/alphageom_project/alpha_geo_processed"
dataset_dir = "/home/mmordig/reinforcement/HumbleAttemptAtGeneralAI/geometry_translation/new/runs/verb_dataset"

raw_datasets = load_from_disk(dataset_dir)
dataset = raw_datasets["test"][:100]
for row in dataset["text"]:
    # wrapping_print(row)
    wrapping_print(extract_question_prompt(row) + "END")
    print("Answer")
    wrapping_print(extract_answer(row) + "END")
    break
# question = format_question_answer(question, "")

In [ ]:
import logging
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import AutoPeftModelForCausalLM
from transformers.trainer_utils import get_last_checkpoint
from transformers import pipeline
import gradio as gr
# from gradio.pipelines import load_from_pipeline


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    level=logging.INFO,
)

# output_dir = sys.argv[1]
output_dir = "runs/trl_sft"
model_name_or_path = get_last_checkpoint(output_dir)
logger.info(f"Loading model from '{model_name_or_path}'")
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.padding_side = "right"

logger.info(f"Loaded model")


In [ ]:

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, num_return_sequences=2, num_beams=4, do_sample=True, max_new_tokens=50)

In [ ]:
# # ?pipe

row = dataset["text"][0]
wrapping_print(extract_question_prompt(row))

In [ ]:
# for output in pipe(raw_datasets["test"].to_iterable_dataset()):
#     print(output)
#     break

In [ ]:
from functools import partial
from datasets import Dataset

def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds

to_dataset = lambda iterable_ds: Dataset.from_generator(partial(gen_from_iterable_dataset, iterable_ds), features=iterable_ds.features)

In [ ]:
ds = load_dataset("rotten_tomatoes", split="train", streaming=True)
to_dataset(ds.take(2))[:2], to_dataset(ds.take(2))[:2]

In [ ]:
dataset_dir = "/home/mmordig/reinforcement/datasets/alpha_geo_small"

dataset = load_dataset("csv", data_dir=dataset_dir)#, nrows=10)
next(iter(dataset["train"]))


In [ ]:

# def formatting_func(row):
#     question = row["nl_statement"]
#     answer = row["fl_statement"]
#     return {"text": f"### Question: {question} ### Answer: {answer}"}
# dataset = dataset.map(formatting_func)

def formatting_func_batched(batch):
    questions = batch["nl_statement"]
    answers = batch["fl_statement"]
    return {"text": [f"### Question: {question} ### Answer: {answer}" for (question, answer) in zip(questions, answers)]}

dataset = dataset.map(formatting_func_batched, batched=True)
dataset = dataset.remove_columns(set(dataset["train"].column_names) - {"text"})
dataset = dataset["train"].train_test_split(0.1, seed=1)
dataset_filename = "/home/mmordig/reinforcement/HumbleAttemptAtGeneralAI/geometry_translation/new/runs/verb_dataset"
dataset.save_to_disk(dataset_filename)
dataset, dataset["train"][:2]

In [ ]:
load_dataset(dataset_filename)

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer


dataset_dir = "/home/mmordig/reinforcement/datasets/alpha_geo_small"
model_name_or_path = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side="left")

tokenizer(" ")["input_ids"][0]


In [ ]:
def get_constant_length_dataset(dataset_dir, **kwargs):
    # use with SFTTrainer together with arg packing=True, not compatible with DataCollatorForCompletionOnlyLM
    
    text_dataset = load_dataset("csv", data_dir=dataset_dir, **kwargs)
    # next(iter(text_dataset["train"]))
    
    def formatting_func(row):
        question = row["nl_statement"]
        answer = row["fl_statement"]
        return f"### Question: {question} ### Answer: {answer}"

    dataset = trl.trainer.ConstantLengthDataset(tokenizer, text_dataset["train"], formatting_func=formatting_func)
    # return dataset
    return DatasetDict({
        "train": dataset,
    })
dataset = get_constant_length_dataset(dataset_dir)
res = next(iter(dataset["train"]))
tokenizer.decode(res["input_ids"]).split("### ")